# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [ ]:
import xlwings as xw
import numpy as np

In [ ]:
xw.__version__

In [ ]:
!pip install xlwings

In [ ]:
xw.__version__

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

<Sheet [себестоимостьА_в1.xlsx]Рецептура>

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

True

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
import os 
import pandas as pd
path = "recipes.xlsx"
recipes_excel = xw.Book(path)

reviews_sample = pd.read_csv("reviews_sample.csv", sep=",", parse_dates=['date'])
reviews_sample = reviews_sample.rename(columns={"Unnamed: 0": "index"})
reviews_sample = reviews_sample.set_index("index")
print (reviews_sample)

recipes_sample_with_tags_ingredients = pd.read_csv("recipes_sample_with_tags_ingredients.csv", sep=",", parse_dates=['submitted'])
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients.rename(columns={"Unnamed: 0": "index"})
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients.set_index("index")
recipes_sample_with_tags_ingredients.drop(["contributor_id", "n_steps","n_tags","tags","ingredients"], axis=1, inplace=True)
print (recipes_sample_with_tags_ingredients)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
new_recipes = recipes_sample_with_tags_ingredients.sample(frac = 0.05)
new_reviews = reviews_sample.sample(frac = 0.05) 
print (new_recipes)
print (new_reviews)
recipes_sheet = recipes_excel.sheets.add("Recipes")
reviews_sheet = recipes_excel.sheets.add("Reviews")
recipes_sheet.range("A1").value = new_recipes
reviews_sheet.range("A1").value = new_reviews
recipes_excel.sheets["Sheet1"].delete()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
recipes_sheet.range("J1").value="seconds_assign"
minutes = recipes_sheet.range("D2:D1501").options(np.array).value
seconds = minutes * 60
recipes_sheet.range("J2").options(transpose=True).value = seconds

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
from xlwings.constants import AutoFillType
recipes_sheet.range("L1").value="seconds_formula"
recipes_sheet.range("L2").formula="=D2*60"
recipes_sheet.range("L2").api.AutoFill(recipes_sheet.range("L2:L1501").api, AutoFillType.xlFillDefault)

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
recipes_sheet.range("N1").value="n_reviews"
recipes_sheet.range('N2').formula = '=COUNTIF(Reviews!$C$2:Reviews!$C$6336, "="&Recipes!C2)'
recipes_sheet.range('N2').api.AutoFill(recipes_sheet.range('N2:N1501').api, AutoFillType.xlFillDefault)

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
from xlwings import constants 
recipes_sheet['L1'].font.bold = True
recipes_sheet['J1'].api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
recipes_sheet['J1'].font.bold = True
recipes_sheet['L1'].api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
recipes_sheet['N1'].font.bold = True
recipes_sheet['N1'].api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
recipes_sheet.range('L1').font.bold = True

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:

def color_of_minutes(minutes):
    if (minutes < 5):
        R = 0
        G = 255
        B = 0


    elif (5 <= minutes <= 10):
        R = 255
        G = 255
        B = 0


    elif (minutes > 10):
        R = 255
        G = 0
        B = 0
    return (R,G,B)


for v in recipes_sheet.range("D2:D1501"): 
    recipes_sheet[v.get_address()].color = color_of_minutes(v.value)
    

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate(sheet):
    sheet.range("G1").value = "Task 8"
    sheet.range('G2').formula = '=COUNTIF(Recipes!$C$2:Recipes!$C$1501,"="&C2)>0'
    sheet.range('G2').api.AutoFill(sheet.range(f'G2:G6337').api, AutoFillType.xlFillDefault)
    for i in range(2, len(new_reviews) + 2):
        rating = sheet.range(f'E{i}').value
        if (rating < 0 or rating > 5) or sheet.range(f'G{i}').value == False:
            sheet.range(f'A{i}:F{i}').color = (255, 0, 0)
    sheet.range("G:G").api.Delete()

validate(reviews_sheet)

recipes_excel.save("recipes.xlsx")
recipes_excel.close()